## Predictions of BTC-USD Price Using SBi-LSTM and SBi-GRU

In [ ]:
# load all functions
from C01_visualization import *
from C02_model_predictions import *

In [ ]:
# lib manipulation data
import numpy as np
import pandas as pd

# lib data visualizations
import seaborn as sns
import matplotlib.pyplot as plt

# lib data preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

- config models

In [ ]:
# set random number
import tensorflow as tf
tf.random.set_seed(1234)

### 1. Data Collections

In [ ]:
# load dataset
dataset = pd.read_csv("../dataset/Cryptocurrency-BTC-USD-2024-04.csv", parse_dates=['Date'])
dataset = dataset[["Date", "Open", "High", "Low", "Close"]]
dataset.info()

In [ ]:
# show dataset
print(dataset.tail())

### 2. Exploration Data Analysis

In [ ]:
timeseries_matplotlib(dataset, ["Open", "High", "Low", "Close"])

### 3. Data Preprocessing

In [ ]:
# process feature selection
data = dataset.filter(['Close'])
data = data.values

In [ ]:
# results feature selection
np.round(data[:5],6)

In [ ]:
# process normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(np.array(data).reshape(-1,1))

In [ ]:
# results normalize features
np.round(scaled[:5],6)

In [ ]:
# results normalize features
lineplot_matplotlib1(
  x1=dataset["Date"],
  y1=scaled,
  label1="Close Price"
)

In [ ]:
# process splitting data
train_data, test_data = train_test_split(scaled, train_size=0.80, test_size=0.20, shuffle=False)

In [ ]:
# results splitting data
lineplot_matplotlib2(
  x1=dataset.iloc[0:len(train_data),0], y1=train_data, label1="Training data",
  x2=dataset.iloc[len(train_data):len(dataset),0], y2=test_data, label2="Testing data",
)

### 4. Supervised Learning

In [ ]:
# function for supervised learning
def create_dataset(look_back, dataset):
    
  # declare variable X and Y
  dataX = []
  dataY = []
  
  # for loop for create supervised learning
  for i in range(look_back, len(dataset)):
    dataX.append(dataset[i-look_back:i, 0])
    dataY.append(dataset[i, 0])
      
  # return value X and Y
  return np.array(dataX), np.array(dataY)
# ----------------------------------------------------------------------------------------

In [ ]:
# process supervised learning
x_train, y_train = create_dataset(60, train_data)
x_test, y_test = create_dataset(60, test_data)

In [ ]:
# reshape input to be [samples, time steps, features]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
print(x_train.shape)

In [ ]:
# reshape input to be [samples, time steps, features]
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
print(x_test.shape)

### 5. Algorithms SBi-LSTM-RNN

In [ ]:
# process predict with LSTM
lstm_model = get_models(algorithms="SBi-LSTM-RNN", timestep=x_train.shape[1])

In [ ]:
# results predict with LSTM
lstm_history, lstm_predictions = get_predictions(
  model=lstm_model,
  x_train=x_train, y_train=y_train,
  x_test=x_test, y_test=y_test,
)

In [ ]:
lineplot_matplotlib2(
  x1=dataset["Date"].iloc[len(y_train)+120:], y1=y_test.reshape(-1,1), label1="actual data",
  x2=dataset["Date"].iloc[len(y_train)+120:], y2=lstm_predictions.reshape(-1,1), label2="results predictions"
)